conda activate contest-by

conda install -c conda-forge pandas
conda install -c conda-forge pandas-profiling
conda install -c conda-forge pygwalker


사용자
    ㄴ ts로 구성된 폴더
        ㄴ항목별 폴더
            ㄴ 항목별 분당 센서 데이터 (csv)
        ㄴlabel 데이터 - 이것만 쓰면 될듯? 

In [29]:
import pandas as pd 
import datetime
from datetime import datetime as dt
import os


def _remove_ds_store(dir_list: list):
    return [d for d in dir_list if d != '.DS_Store']


# read data
def get_user_data(user_dir_path: str):
    user_dir_list = _remove_ds_store(os.listdir(user_dir_path))
    user_data_df = pd.DataFrame()
    for user_dir in sorted(user_dir_list):
        try:
            user_day_data_path = os.path.join(user_dir_path, user_dir, user_dir+'_label.csv')
            user_data_df = pd.concat([user_data_df, pd.read_csv(user_day_data_path)])
        except Exception as e:
            print(e)
            print(os.path.join(user_dir_path, user_dir, user_dir+'_label.csv'))

    return user_data_df


# data preprocessing
def processed_user_data(user_data_df: pd.DataFrame):
    # 1. ts기준이 utc 기준이 맞는지 한번더 확인이 필요
    user_data_df['ts'] = user_data_df['ts'].astype(int)
    user_data_df['dt'] = pd.to_datetime(user_data_df['ts'], unit='s')
    user_data_df['dt_kst'] = user_data_df['dt'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
    user_data_df['date_ymd'] = user_data_df['dt_kst'].dt.date
    user_data_df['year'] = user_data_df['dt_kst'].dt.year
    user_data_df['month'] = user_data_df['dt_kst'].dt.month
    user_data_df['day'] = user_data_df['dt_kst'].dt.day
    user_data_df['day_of_week'] = user_data_df['dt_kst'].dt.dayofweek
    user_data_df['day_name'] = user_data_df['dt_kst'].dt.day_name()
    user_data_df['hour'] = user_data_df['dt_kst'].dt.hour
    user_data_df['meridiem'] = user_data_df['dt_kst'].dt.strftime('%p')
    user_data_df['is_weekend'] = user_data_df['dt_kst'].dt.dayofweek.isin([5, 6])
    # user_data_df['is_holiday'] = .weekday()>=5, axis=1)
    del user_data_df['ts'], user_data_df['dt']
    user_data_df.sort_values(['dt_kst'], inplace=True)
    return user_data_df


def get_user_dir_path():
    dir_by_users = [os.path.join(INPUT_PATH, d) for d in os.listdir(INPUT_PATH) if os.path.isdir(d) and 'user' in d]
    sub_dir_by_users = [os.path.join(d, sub_d) for d in dir_by_users for sub_d in _remove_ds_store(os.listdir(d))]
    return sub_dir_by_users


"""
1. 입/출력 경로 설정
"""
INPUT_PATH = './'
OUTPUT_PATH = './outputs'
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)
    print('{} 생성완료'.format(OUTPUT_PATH))


"""
2. 사용자별 데이터 파일(.csv) 생성
"""
user_dir_path_list = get_user_dir_path()
for user_dir_path in user_dir_path_list:
    # print(user_dir_path)
    user_data_df = get_user_data(user_dir_path)
    processed_user_data_df = processed_user_data(user_data_df)

    # path 변경시 수정 필요
    user_id = user_dir_path.split('/')[-1]
    processed_user_data_df['user_id'] = user_id
    save_file_name = user_id+'_data.csv'
    processed_user_data_df.to_csv(os.path.join(OUTPUT_PATH, save_file_name), index=False)
    print('{} 데이터 저장 완료!'.format(user_id))
    

"""
3. 사용자 전체 데이터 파일(.csv) 생성
"""
save_all_user_data_file_name = 'all_users_data.csv'
all_user_data_df = pd.DataFrame()
pass_file_name_list = []
for user_output_file_name in os.listdir(OUTPUT_PATH):
    if user_output_file_name != save_all_user_data_file_name:
        try:
            user_output_file_path = os.path.join(OUTPUT_PATH, user_output_file_name)
            all_user_data_df = pd.concat([all_user_data_df, pd.read_csv(user_output_file_path)])
            pass_file_name_list.append(user_output_file_path)
        except:
            pass

# print(sorted(pass_file_name_list))
all_user_data_df.drop_duplicates(inplace=True)
all_user_data_df.sort_values(['user_id', 'dt_kst'], inplace=True)
all_user_data_df.to_csv(os.path.join(OUTPUT_PATH, save_all_user_data_file_name), index=False)
print('전체 사용자 데이터 저장 완료!')

[Errno 2] No such file or directory: './user26-30/user30/1599354900/1599354900_label.csv'
./user26-30/user30/1599354900/1599354900_label.csv
[Errno 2] No such file or directory: './user26-30/user30/1599873300/1599873300_label.csv'
./user26-30/user30/1599873300/1599873300_label.csv
[Errno 2] No such file or directory: './user26-30/user30/1599959700/1599959700_label.csv'
./user26-30/user30/1599959700/1599959700_label.csv
[Errno 2] No such file or directory: './user26-30/user30/1600038900/1600038900_label.csv'
./user26-30/user30/1600038900/1600038900_label.csv
[Errno 2] No such file or directory: './user26-30/user30/1600386000/1600386000_label.csv'
./user26-30/user30/1600386000/1600386000_label.csv
user30 데이터 저장 완료!
user26 데이터 저장 완료!
user28 데이터 저장 완료!
user29 데이터 저장 완료!
user27 데이터 저장 완료!
user08 데이터 저장 완료!
[Errno 2] No such file or directory: './user07-10/user09/1600997940/1600997940_label.csv'
./user07-10/user09/1600997940/1600997940_label.csv
[Errno 2] No such file or directory: './user07